# 1. Parametric Model

A parametric model can represent a class of instances where each is defined by a value of the parameters. Examples include lines, or circles, or even a parametrized template.

## (1) Fitting a parametric model

- Choose a parametric model to represent a set of features.
- Membership criterion is not local:
    Can't tell whether a point in the image belongs to a given model just by looking at that point.
- Computational complexity is important:
    Not feasible to examine possible parameter setting.
    
## (2) Difficulty of line fitting.

Because edge images is a collection of pixels.

- Extra edge points (clutter), multiple models.
- Only some parts of each line detected, and some parts are missing.
- Noise in measured edge points, orientations.

<img src="images/6/line_fitting.png" width="300px">

## (3) Voting

It is a general technique where we lete the features vote for all models that are pompatible with it. 

1. Cycle through features, each casting votes for model parameters.
2. Look for model parameters that receive a lot of votes.

#### 1) Why it works?:
- Noise and clutter features will cast votes too, but typically their votes should be inconsistent with the majority of __good__ features.
- Ok if some features not observed, as model can span multiple fragments.

## (4) Fitting lines using Hough algorithm

To fit lines, we need to answer a few questions:
- Given points that belong to a line, what is the line?
- How many lines are there?
- Which points belong to which lines?

A possible method for those question would be __Hough Transform__, which is a voting technique.

#### 1) Main idea:
1. Each edge point votes for compatible lines.
2. Look for lines that get many votes.

If we keep track of which points voted for which lines, this will able to go back and be said which points belong to that line.

#### 2) Hough space

In below example, there are two parameters $m, b$ which draw a line.

<img src="images/6/Hough_space1.png" width="450px">

If there is a line in an image space, a point can be represented in Hough space,

<img src="images/6/Hough_space2.png" width="450px">

The procedure of above picture is as follow,

1. A point in an image space is a line in Hough space.
<img src="images/6/Hough_space3.png" width="450px">

2. An another point in an image space will be represented as an another line in Hough space.
<img src="images/6/Hough_space4.png" width="450px">

3. A point intercepting the two lines in Hough space would be consistent with both points in an image space, because that's the $m$ and $b$ that's consistent with being on a line that goes through $x_0, y_0$ and on a line that goes through $m_1, b_1$. 
<img src="images/6/Hough_space5.png" width="450px">

#### 3) Hough algorithm in Cartasian coordinate

So, this is how to find lines from points. 

- Let each edge point in image space vote for a set of possible parameters in Hough space.
<img src="images/6/Hough_algorithm1.png" width="450px">

- Accumulate votes in discrete set of bins; parameters with the most votes indicate line in image space.
<img src="images/6/Hough_algorithm2.png" width="450px">

However, in the method which represents Hough algorithm in Cartesian coordinate, vertical and horizontal line will be infinity in Hough space. This is a really big problem! However, __using polar coordiante__ for Hough algorithm can resolve this issue.

#### 4) Polar representation for lines

<img src="images/6/Polar.png" width="450px">

The purple line is going to be defined by two quatities:
- distance of a line $d$: 
    - this is the distance of a line to the origin
    - the perpendicular distance
    - the distance to the closest point on the line to the origin
- angle $\theta$:
    - angle from the perpendicular $d$ to x-axis or y-axis (that doesn't matter).
    
A point in image space is now a sinusoid in Hough space. 

Before moving to next, it has to noted that there's a redundancy or an ambiguity here. 
- If $d$ can only be positive, it is able to spin all the way around ($\theta$ would have to go from 0 to $2\pi$, 0 to 360 degrees).
- If $d$ can be positive or negative, then $\theta$ has to go from 0 to $\pi$ or 0 to $-\pi$.

#### 5) Hough algorithm in polar coordinate

So, polar representation is used for the polar parameterization of the line. And Hough Accumulator Array is just a funcy word for the thing that's going to collect the votes.

<img src="images/6/Hough_algorithm3.png" width="450px">

The number of $d$ and $\theta$ in Hough Accumulator Array varies in different initial setting.

Then, the Hough algorithm follows below rule,

<img src="images/6/Hough_algorithm4.png" width="450px">

#### 6) Complexity of the Hough transform

Whenever implementing the algorithm, we need to think about things like how well does it work? More importantly for algorithms, we have to talk about things like complexity.

- Space complexity: how much memory do I have to use? 
    - $k^n$ bins is needed ($n$ dimensions, $k$ bins each).
    - Adding the number of parameters, which increases $n$ can be very expensive in terms of memory.
- Time complexity in terms of number of voting elements:
    - Voting is linearly proportional to the number of edge points.
    - The time complexity is constant in the number of features or edge points that have been detected.
    
#### 7) Hough examples

The brightest point is where the line is.

<img src="images/6/example1.png" width="450px">

<img src="images/6/example1.png" width="450px">